# Анализ продаж пункта выдачи заказов

# Описание проекта

Анализирую набор данных по продажам пункта выдачи заказов (ПВЗ) департмент стора.

Его сотрудники столкнулись с проблемой высокой загрузки ПВЗ.

Увеличение времени обслуживания клиентов приводит к ухудшению качества клиентского сервиса.

Ранее выяснилось, что время ожидания на ПВЗ выросло из-за нехватки примерочных.

Быстро произвести реорганизацию пространства ПВЗ и построить в 3 раза больше примерочных не видится возможным, а снизить количество недовольных клиентов требуется уже сейчас.

Руководители ПВЗ высказали предположение, что постамат или наличие дополнительного места для примерки обуви — один из быстрых способов решения проблемы.

Необходимо проверить это предположение на цифрах.

Разделю весь товар на три группы: нужна примерка в примерочной, легко примерить в зале ожидания, примерка не нужна. Проведу группировку заказов следующим образом:
- Нужна примерка в примерочной - все заказы, где хотя бы одна позиция одежда (_ktt2 = Одежда)
- Легко примерить в зале - заказы, в которых только обувь и ничего больше (_ktt2 = Обувь). Заказы вида Обувь+Услуги подходят тоже
- Примерка не нужна - заказы, в которых только сумки (_ktt2 = Сумки). Заказы вида Сумки+Услуги подходят тоже

# Задание 1

Посмотрю по месяцам какую долю от общего числа заказов составляют заказы каждой из предложенных групп.

Рассматриваю подтвержденные заказы (заказ определяется по Journal_id).

Полученный результат представлю в виде таблицы и сделаю выводы.

# Задание 2

Посмотрю по месяцам количество заказов группы "Легко примерить в зале" по количеству заказанных позиций. Рассматриваю подтвержденные заказы.

Группировка вида: 
- однотоварник - 1 позиция, 
- двухтоварних - 2 единицы в заказе и т.д.

Услуги (_ktt2=Услуги) за заказанную единицу не считаются и будут исключены.

Разложу результат по месяцам и представлю в виде таблицы.

В дополнение посчитаю выкупаемость (Approve to Net: Net_USD/_Approved_USD) в той же разбивки (по месяцам и по типам корзин). Результат представлю в виде таблицы и сделаю выводы.

# Структура проекта

### 1. Изучение общей информации
### 2. Предобработка данных
### 3. Задание 1
### 4. Задание 2

## 1. Изучение общей информации

### 1.1 Загрузка библиотек

In [1]:
import pandas as pd
import numpy as np

### 1.2 Чтение файла

In [2]:
try:
    data = pd.read_excel("/Users/angelinazigalova/Desktop/data_collect.xlsx" , sheet_name = "data")
except:
    print('Не удалось открыть файл')

### 1.3 Просмотр общей информации о данных

In [3]:
data.head(5)

,Week,Day,DelivType,PymentType,Journal_id,_ktt1,_ktt2,_ktt3,_itemid,_colorid,_Approved_USD,_Approved_PCS,_Net_USD,_Net_PCS
0,Week 17 (23.11.20-29.11.20),2020-11-27,ЦУМ - Самовывоз,Prepaid,EO01386067,Товары для мужчин,Аксессуары из кожи,Ремни,7625454,Черный_BLACK,360.803788,1,0.000000,0
1,Week 23 (04.01.21-10.01.21),2021-01-09,ЦУМ - Самовывоз,Postpaid,EO01496554,Товары для мужчин,Аксессуары из кожи,Ремни,7625454,Черный_BLACK,240.875404,1,240.875404,1
2,Week 25 (18.01.21-24.01.21),2021-01-24,ЦУМ - Самовывоз,Postpaid,EO01538172,Товары для мужчин,Аксессуары из кожи,Ремни,7625454,Черный_BLACK,249.103035,1,249.103035,1
3,Week 2 (08.02.21-14.02.21),2021-02-10,ЦУМ - Самовывоз,Postpaid,EO01586470,Товары для мужчин,Аксессуары из кожи,Ремни,7625454,Черный_BLACK,222.563256,1,222.563256,1
4,Week 16 (16.11.20-22.11.20),2020-11-21,ЦУМ - Самовывоз,Postpaid,EO01373837,Товары для мужчин,Бижутерия,Браслет,7995454,Черный_LK0001,544.009963,1,0.000000,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525466 entries, 0 to 525465
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Week           525466 non-null  object        
 1   Day            525466 non-null  datetime64[ns]
 2   DelivType      525466 non-null  object        
 3   PymentType     525466 non-null  object        
 4   Journal_id     525466 non-null  object        
 5   _ktt1          525466 non-null  object        
 6   _ktt2          525466 non-null  object        
 7   _ktt3          525466 non-null  object        
 8   _itemid        525466 non-null  int64         
 9   _colorid       421490 non-null  object        
 10  _Approved_USD  525466 non-null  float64       
 11  _Approved_PCS  525466 non-null  int64         
 12  _Net_USD       525466 non-null  float64       
 13  _Net_PCS       525466 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(3), object(8

В столбце с кодом цвета есть пропуски - можно заполнить.

Также для удобства нужно привести названия столбцов к единому виду.


## 2. Предобработка данных

###  2.1 Поиск явных дубликатов

In [5]:
print('Количество найденых дубликатов:', data.duplicated().sum())

Количество найденых дубликатов: 0


### 2.2 Привожу столбцы к единому виду

In [6]:
data.columns = ['week', 'day', 'deliv_type', 'pyment_type', 'journal_id', 'ktt1', 'ktt2', 'ktt3', 'item_id', 'color_id', 'approved_usd', 'approved_pcs', 'net_usd', 'net_pcs']

### 2.3 Заменяю пропуски в столбце с кодом цвета

In [7]:
data["color_id"] = data["color_id"].fillna("Неизвестный")

### 2.4 Создаю новый столбец с группой товара

In [8]:
data['group'] = np.where(data.groupby('journal_id')['ktt2'].transform(lambda x: 'Одежда' in x.values), 'Нужна примерка в примерочной',
                          np.where(data.groupby('journal_id')['ktt2'].transform(lambda x: all(item in x.values for item in ['Сумки', 'Услуги']) and len(x) == 2), 'Примерка не нужна',
                                   np.where(data.groupby('journal_id')['ktt2'].transform(lambda x: 'Обувь' in x.values), 'Легко примерить в зале',
                                            np.where(data['ktt2'].isin(['Сумки', 'Сумки и Услуги']), 'Примерка не нужна', 'другое'))))

### 2.5 Создаю новый столбец с месяцем заказа

In [9]:
data['month'] = data['day'].dt.month

### 2.6 Фильтрую данные по подтвержденному заказу

In [10]:
data = data[data['approved_pcs'] > 0]

## 3. Задание 1

In [11]:
# Для каждого месяца и группы считаю количество заказов
pivot_table = pd.pivot_table(data, index = ['month', 'group'], values = 'journal_id', aggfunc = 'count').reset_index()
# Считаю количество заказов для каждого месяца
pivot_table['journal_id_month'] = pivot_table.groupby('month')['journal_id'].transform('sum')
# Создаем столбец с долей от общего числа заказов
pivot_table['part_of_the_total'] = round(pivot_table['journal_id'] / pivot_table['journal_id_month'], 2)

In [12]:
pivot_table

,month,group,journal_id,journal_id_month,part_of_the_total
0,1,Легко примерить в зале,8249,40444,0.20
1,1,Нужна примерка в примерочной,26033,40444,0.64
2,1,Примерка не нужна,1458,40444,0.04
3,1,другое,4704,40444,0.12
4,2,Легко примерить в зале,6540,36447,0.18
5,2,Нужна примерка в примерочной,23105,36447,0.63
6,2,Примерка не нужна,1389,36447,0.04
7,2,другое,5413,36447,0.15
8,3,Легко примерить в зале,5284,28065,0.19
9,3,Нужна примерка в примерочной,17702,28065,0.63


Из таблицы можно сделать выводы:
- Заказы, для которых примерка не нужна, крайне малочисленны - 3-4% от общего числа заказов в месяц.
- Заказы, которые можно померять в зале каждый месяц составляют от 16 до 21% от всех заказов.
- Заказы для которых необходима примерочная, стабильно составляют 50-60% от всех заказов.

Предположение о дополнительном месте в зале для примерки обуви можно подтвердить -  это разгрузит примерочные на 20%.

## 4. Задание 2

### 4.1 Фильтрую данные по группе

In [16]:
data_new = data[data['group'] == 'Легко примерить в зале']

### 4.2 Фильтрую данные по услуге

In [17]:
data_new = data_new[data_new['ktt2'] != 'Услуги']

### 4.3 Создаю новую группу товаров по количеству подтвержденных товаров в заказе

In [18]:
# Проверяю распределение количества товаров в заказе
data_new['approved_pcs'].value_counts()

1    71812
2     8622
3      891
4       94
6       24
5       21
9        7
7        1
Name: approved_pcs, dtype: int64

Максимум позиций в заказе - 9. Более трех товаров в заказе встречается крайне редко, поэтому объединю их в одну группу.

In [19]:
# Напишу функцию дляопределения групп
def my_function(row):
    if row == 1:
        return 'Однотоварник'
    if row == 2:
        return   'Двухтоварник'
    if row == 3:
        return 'Трехтоварник'
    if row > 3:
        return 'Более 3 товаров'

In [20]:
# Создаю новый столбец
data_new['pcs_group'] = data_new['approved_pcs'].apply(my_function)

### 4.4 Создаю таблицу в разбивке по месяцам

In [21]:
# Считаю оккупаемость
data_new['approve_to_net'] = data_new['net_usd'] / data_new['approved_usd']

In [22]:
table = data_new.pivot_table(index = ['month', 'pcs_group'], values = 'journal_id', aggfunc = 'count').reset_index()
table_new = data_new.pivot_table(index = ['month', 'pcs_group'], values = 'approve_to_net', aggfunc = 'mean').reset_index()
merged_data = pd.merge(table, table_new, on=['month', 'pcs_group'])

In [23]:
merged_data

,month,pcs_group,journal_id,approve_to_net
0,1,Более 3 товаров,16,0.312500
1,1,Двухтоварник,745,0.485906
2,1,Однотоварник,7418,0.504988
3,1,Трехтоварник,70,0.528571
4,2,Более 3 товаров,10,0.700000
5,2,Двухтоварник,616,0.493506
6,2,Однотоварник,5853,0.502307
7,2,Трехтоварник,61,0.459016
8,3,Более 3 товаров,13,0.769231
9,3,Двухтоварник,708,0.504237


Исходя изтаблицы, можно сделать вывод:
- Однотоварники - это наиболеемногочисленная группа. Окупаемость для группы от 48 до 50 % каждый месяц.
- Двух товарников меньше, процент окупаемости от 47 до 53 %.
- Для трехтоварников процент окупаемости от 45 до 63 %
- Для самой малочисленной группы, где количество товаров более 3, процент выкупа до 76 %.

Тендеция выкупа говорит о том, что чем больше товара в корзине, тем больше процент выкупаемости. Однако, для группы "более 3 товаров" данных слишком мало.

Посмотрю на выкупаемость по группам за все время

In [26]:
new = table_new.pivot_table(index = 'pcs_group', values = 'approve_to_net', aggfunc = 'mean').reset_index().sort_values(by = 'approve_to_net')
new['approve_to_net'] = round(new['approve_to_net'], 2)
new

,pcs_group,approve_to_net
2,Однотоварник,0.50
1,Двухтоварник,0.50
0,Более 3 товаров,0.51
3,Трехтоварник,0.54


Думаю, что можно подтвердить  гипотезу, что с ростом количества товара в корзине - растет и процент выкупа